# Réseaux convolutionnels pour le traitement de l'image  


## Partie B : Data Augmentation, Fine Tuning

Vincent Guigue (vincent.guigue@agroparistech.fr), à partir des supports de :<BR>
Nicolas Baskiotis (nicolas.baskiotis@soronne-univeriste.fr) Benjamin Piwowarski (benjamin.piwowarski@sorbonne-universite.fr) -- MLIA/ISIR, Sorbonne Université

In [2]:
from tqdm import tqdm
import numpy as np
import torch
from torch import nn
from torch.nn import functional as F
from torch import optim
import torchvision
from torchvision import transforms
from torch.utils.tensorboard import SummaryWriter
import time
import os
from tensorboard import notebook
from tensorboard import notebook
from torch.utils.data import TensorDataset, DataLoader,Dataset
import matplotlib.pyplot as plt

In [3]:
TB_PATH = "/tmp/logs/module2"
%load_ext tensorboard
%tensorboard --logdir {TB_PATH}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

print(device)

Reusing TensorBoard on port 6007 (pid 47637), started 17:29:16 ago. (Use '!kill 47637' to kill it.)

mps


On reprend le code du TP précédent. 

In [ ]:
TRAIN_BATCH_SIZE = 256
TEST_BATCH_SIZE = 512

def accuracy(yhat,y):
    # y encode les indexes, s'assurer de la bonne taille de tenseur
    assert len(y.shape)==1 or y.size(1)==1
    # return (torch.argmax(yhat,1).view(y.size(0),-1)== y.view(-1,1)).float().mean() # not working on M1
    return (torch.max(yhat,1).indices.view(y.size(0),-1)== y.view(-1,1)).float().mean()
#trick pour les mac M1 => Pb avec argmax
    



def train(model,epochs,train_loader,test_loader):
    writer = SummaryWriter(f"{TB_PATH}/{model.name}")
    # params = []
    # Permet de ne prendre en compte que les paramètres 
    # à entrainer (requires_grad==True) pour l'optimiseur. Pas obligatoire.
    # for param in model.parameters():
    #     if param.requires_grad:
    #         params.append(param)
    params = model.parameters()
    optim = torch.optim.Adam(params,lr=1e-3)
    model = model.to(device)
    print(f"running {model.name}")
    loss = nn.CrossEntropyLoss()
    for epoch in tqdm(range(epochs)):
        cumloss, cumacc, count = 0, 0, 0
        model.train()
        for x,y in train_loader:
            optim.zero_grad()
            x,y = x.to(device), y.to(device)
            yhat = model(x)
            l = loss(yhat,y)
            l.backward()
            optim.step()
            cumloss += l*len(x)
            cumacc += accuracy(yhat,y)*len(x)
            count += len(x)
        writer.add_scalar('loss/train',cumloss/count,epoch)
        writer.add_scalar('accuracy/train',cumacc/count,epoch)

        if epoch % 1 == 0:
            model.eval()
            with torch.no_grad():
                cumloss, cumacc, count = 0, 0, 0
                for x,y in test_loader:
                    x,y = x.to(device), y.to(device)
                    yhat = model(x)
                    cumloss += loss(yhat,y)*len(x)
                    cumacc += accuracy(yhat,y)*len(x)
                    count += len(x)
                writer.add_scalar(f'loss/test',cumloss/count,epoch)
                writer.add_scalar('accuracy/test',cumacc/count,epoch)

def compute_accuracy(model, datal):
    with torch.no_grad():
        cumloss, cumacc, count = 0, 0, 0
        for x,y in datal:
            x,y = x.to(device), y.to(device)
            yhat = model(x)
            cumacc += accuracy(yhat,y)*len(x)
            count += len(x)
    return cumacc/count

def set_parameter_requires_grad(model,b):
    for p in model.parameters():
        p.requires_grad = b

def getSaliency(model,img,label):
    model.zero_grad()
    img = img.to(device)
    img.requires_grad = True
    img.grad = None
    outputs = nn.Softmax(dim=1)(model(img.unsqueeze(0)))
    output=outputs[0,label] 
    output.backward()
    sal=img.grad.abs()
    if sal.dim()>2:
        sal=torch.max(sal,dim=0)[0]
    fig=plt.figure(figsize=(8, 8))
    fig.add_subplot(1, 2, 1)
    plt.imshow(img.detach().cpu().permute(1,2,0),cmap="gray")
    fig.add_subplot(1, 2, 2)
    plt.imshow(sal.to('cpu'),cmap="seismic",interpolation="bilinear")
    plt.show()
    return sal
        
def generate_cam(model,input_image,target_class=None):
    ## Calcul du forward sur l'image
    with torch.no_grad():
        input_image=input_image.to(device)
        x = model.features(input_image)
        out=model(input_image)
        out=torch.nn.functional.softmax(out,-1)
    if target_class is None:
        target_class = torch.max(out,dim=-1)[1].item()
    print("target_class",target_class)
    ## Récupération des poids du linéaire
    weights = list(model.classifier._modules.values())[-1].weight.data  
    fig = plt.figure(figsize=(16, 8))
    fig.add_subplot(1,2, 1)
    img=input_image.to("cpu") ##*torch.tensor(std).view(3,1,1)+torch.tensor(mean).view(3,1,1)
    img=torch.nn.functional.interpolate(img, size=(244, 244), mode="bilinear", align_corners=False)
    plt.imshow(img.cpu().squeeze(0).permute(1,2,0))
    ## Calcul de CAM
    y=x*weights[target_class].view(1,-1,1,1)
    y=(y.sum(1))  
    fig.add_subplot(1, 2, 2)
    y=torch.nn.functional.interpolate(y.unsqueeze(0),size=(244,244),mode="bilinear",align_corners=False)
    plt.imshow(y.cpu().squeeze(),cmap="bwr",vmax=4,vmin=-4)
    plt.show()

def analyse_conv(model,img,nb_filtres=16):
    x  = img.unsqueeze(0).to(device)
    img_conv = []
    img_pool = []
    for m in model.features._modules.values():
        x = m.forward(x)
        if isinstance(m,nn.Conv2d):
            img_conv.append((x.squeeze(0),m.weight))
        if isinstance(m,nn.MaxPool2d) or isinstance(m,nn.AvgPool2d):
            img_pool.append(x.squeeze(0))
    plt.figure()
    plt.imshow(img.permute(1,2,0).to('cpu'),cmap='gray')
    # nombre de filtres
    ksmax = min(nb_filtres, max([p[0].size(0) for p in img_conv]))
    fig, axs = plt.subplots(3*len(img_conv),ksmax,figsize=(20,5))
    for i,((img_c,w),img_p) in enumerate(zip(img_conv,img_pool)):
        for j in range(min(nb_filtres,img_c.size(0))):
            axs[3*i,j].imshow(np.array(w[j,0].to('cpu').detach()),cmap="gray")
            axs[3*i+1,j].imshow(np.array(img_c[j].to('cpu').detach()),cmap="gray")                             
        for j in range(min(nb_filtres,img_p.size(0))):
            axs[3*i+2,j].imshow(np.array(img_p[j].to('cpu').detach()),cmap="gray")
    plt.show()

In [4]:
import os

def save_model(model,fichier): # pas de sauvegarde de l'optimiseur ici
      """ sauvegarde du modèle dans fichier """
      state = {'model_state': model.state_dict()}
      torch.save(state,fichier) # pas besoin de passer par pickle
 
def load_model(fichier,model):
      """ Si le fichier existe, on charge le modèle  """
      if os.path.isfile(fichier):
          state = torch.load(fichier)
          model.load_state_dict(state['model_state'])

# Données CIFAR

La base de données CIFAR10  contient  60000 images couleur (RGB) 32x32 pixels. Les images appartiennent à 10 catégories (6000 images par classe): 'plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship' et 'truck'. Le
dataset est composé de 50000 exemples d'apprentissage et 10000 de test (170Mo).


In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
    ])

batchsize = 128              

cifar_trainset = torchvision.datasets.CIFAR10(root='/tmp/data', train=True, download=True, transform=transform)
cifar_train_loader = torch.utils.data.DataLoader(cifar_trainset, batch_size=batchsize, pin_memory=True, shuffle=True)
cifar_testset = torchvision.datasets.CIFAR10(root='/tmp/data', train=False, download=True, transform=transform)
cifar_test_loader = torch.utils.data.DataLoader(cifar_testset, batch_size=batchsize, pin_memory=True, shuffle=False)
print(cifar_trainset.classes)

* Testez un réseau convolutionnel de deux couches avec 32 filtres et un réseau linéaire type *Linear(in_dim,120)->ReLU->Linear(120,80)->Relu->Linear(80,10)*  sur cette base de données et comparez les résultats. 
* Expérimentez également d'autres architectures de convolution (nombre de filtres, taille des filtres, différents strides, éventuellement padding). 
* Comparez le nombre de paramètres des réseaux
* Visualisez la carte de saillance et les filtres du réseau.
* Visualisez la CAM du réseau

In [5]:
## Définition du réseau à compléter
class ConvCIFAR(nn.Module):
    def __init__(self,nb_channels=32):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=nb_channels, kernel_size=5)
        self.pool1 = nn.MaxPool2d(kernel_size=3, stride=1)
        self.conv2 = nn.Conv2d(in_channels=nb_channels, out_channels=nb_channels, kernel_size=5)
        self.pool2 = nn.MaxPool2d(3, 1)
        torch.nn.init.xavier_uniform_(self.conv1.weight)
        torch.nn.init.xavier_uniform_(self.conv2.weight)
        self.features = nn.Sequential(self.conv1,nn.ReLU(),self.pool1,self.conv2,nn.ReLU(),self.pool2)
        self.classifier = nn.Sequential(nn.Linear(20*20*nb_channels,120),nn.ReLU(),nn.Linear(120,80),nn.ReLU(),nn.Linear(80,10))
        

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x.view(x.size(0), -1))
        return x

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


In [ ]:
## Entraînement du réseau convolutionnel, affichage du nombre de paramètres de chaque réseau

# 1. instanciation (idéalement en jouant avec un paramètre sur la taille des convolutions)
# 2. nommage
# 3. train
# 4. comptage des paramètres

## <CORRECTION>
CIFARNet = ConvCIFAR(32).to(device)
CIFARNet.name = "cifar-"+time.asctime()
train(CIFARNet,30,cifar_train_loader,cifar_test_loader)
print("#Params CIFAR conv", count_parameters(CIFARNet))
## </CORRECTION>

In [6]:
## Définition du réseau fully-connected et entraînement

# seqNet = nn.Sequential(... Réseau linéaire)
# nommage
# entrainement

## <CORRECTION>
seqNet = nn.Sequential(nn.Flatten(1),nn.Linear(3*32*32,500), nn.ReLU(), nn.Linear(500,100),nn.ReLU(),nn.Linear(100,10)).to(device)
seqNet.name = "cifarSeqNet-"+time.asctime()
print("#Params SeqNet ",count_parameters(seqNet))

train(seqNet,30,cifar_train_loader,cifar_test_loader)
## </CORRECTION>

#Params SeqNet  1587610


NameError: name 'train' is not defined

In [7]:
# sauvegarde / chargement

path = "./model/"

# fichier = path+f"CIFARNet"
# save_model(CIFARNet,fichier)

# fichier = path+f"cifarSeqNet"
# save_model(seqNet,fichier)

# vous pouvez utiliser les formules symmétriques pour le chargement

CIFARNet = ConvCIFAR(32).to(device)
CIFARNet.name = "cifar-"+time.asctime()

seqNet = nn.Sequential(nn.Flatten(1),nn.Linear(3*32*32,500), nn.ReLU(), nn.Linear(500,100),nn.ReLU(),nn.Linear(100,10)).to(device)
seqNet.name = "cifarSeqNet-"+time.asctime()

load_model( path+f"CIFARNet", CIFARNet)
load_model( path+f"cifarSeqNet", seqNet)

In [ ]:
# verification des performances

# pour un batch... A la main
# x,y = next(iter(cifar_test_loader))
# x,y = x.to(device), y.to(device)
# yhat = CIFARNet(x)
# print(accuracy(yhat, y))

# pour toutes les données
print("CIFARNet : ", compute_accuracy(CIFARNet, cifar_test_loader))
print("seqNet : ", compute_accuracy(seqNet, cifar_test_loader))

# check:
# CIFARNet :  tensor(0.6593, device='mps:0')
# seqNet :  tensor(0.5103, device='mps:0')

In [8]:
fichier = path+f"CIFARNet"
save_model(CIFARNet.cpu(),fichier)

fichier = path+f"cifarSeqNet"
save_model(seqNet.cpu(),fichier)

In [ ]:
## Analyse des filtres du réseau
## [[student]]
analyse_conv(CIFARNet,cifar_train_loader.dataset[1][0])
## [[/student]]

In [ ]:
## Carte de saillance du réseau
## [[student]]
# inputs,labels=iter(cifar_train_loader) #.next()
inputs,labels=next(iter(cifar_train_loader))
for i in range(10):
    getSaliency(CIFARNet,inputs[i],labels[i])
## [[/student]]

In [ ]:
## CAM du réseau

## [[student]]
set_parameter_requires_grad(CIFARNet,False)
CIFARNet.classifier = nn.Sequential(torch.nn.Unflatten(1,(32,20,20)),torch.nn.AvgPool2d(20,1),torch.nn.Flatten(1),torch.nn.Linear(32,10))
train(CIFARNet,10,cifar_train_loader,cifar_test_loader)
## [[/student]]

In [ ]:
cifar_batch, cifar_class = next(iter(cifar_train_loader))
cifar_features = CIFARNet.features.forward(cifar_batch)
for i in range(10):
    generate_cam(CIFARNet,cifar_batch[i].unsqueeze(0),cifar_class[i])

# Data Augmentation

Pour améliorer les résultats, une technique courante est d'augmenter les données par des variantes des images du corpus. Cela permet de gagner en robustesse vis à vis de diverses transformations en forçant le réseau à apprendre des invariants (e.g. d'échelle, de rotation, d'inversion, de luminosité, etc.). 

Insérez quelques transformations de données lors du chargement des données (la liste des transformations disponibles se trouvent dans <a href=https://pytorch.org/vision/stable/transforms.html> torchvision.transforms</a>, par exemple **RandomHorizontalFlip()**, **RandomResizedCrop()**) et relancez l'apprentissage pour voir l'effet. Les transformations sont à insérer dans le **transforms.Compose()** avant la transformation en tenseur.

In [ ]:
## Définition de la transformation pour Data Augmentation et création du réseau et des dataloader.

transformTrain = transforms.Compose(
    [transforms.RandomHorizontalFlip(), transforms.RandomResizedCrop(32), transforms.ToTensor()#,
    # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])


cifar_trainset_aug = torchvision.datasets.CIFAR10(root='/tmp/data', train=True,
                                        download=False, transform=transformTrain)
cifar_trainloader_aug = torch.utils.data.DataLoader(cifar_trainset_aug, batch_size=batchsize, pin_memory=True,
                                          shuffle=True)

CIFARNet_aug = ConvCIFAR().to(device)
CIFARNet_aug.name = "cifar_aug"+time.asctime()


In [ ]:
## Apprentissage du réseau

train(CIFARNet_aug,30,cifar_trainloader_aug,cifar_test_loader) 


# Modèles pré-entraînés / Transfert

PyTorch propose un certain nombre de modèles pré-entraînés sur le très gros corpus d'images ImageNet. Ces modèles très lourds demandent beaucoup de ressources pour être entraînés efficacement. Mais une fois leur entraînement effectué, ils peuvent être appliqués assez facilement sur d'autres corpus que ImageNet, moyennant quelques adaptations. 

La liste des modèles disponibles est disponible sur <a href=https://pytorch.org/vision/stable/models.html>cette page de documentation</a>. 

Dans la suite nous considérons le modèle <a href=https://pytorch.org/hub/pytorch_vision_alexnet/>AlexNet</a> pour l'extraction de features.

Vous pouvez comparer ensuite les résultats avec d'autres modèles, du type VGG, ResNet etc.

La sortie du réseau doit être adaptée et ré-entraînée pour permettre de classer des images sur notre corpus CIFAR. Les images doivent être également traitées pour correspondre au pré-traitement de AlexNet : en particulier la taille doit être de (224,224) et il faut également les normaliser.

In [ ]:
input_size=224

mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

transformAlexTrain=transforms.Compose([transforms.Resize(input_size),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
transformAlexTest=transforms.Compose([transforms.Resize(input_size),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])

alex_trainset = torchvision.datasets.CIFAR10(root='/tmp/data', train=True,
                                        download=True, transform=transformAlexTrain)
alex_trainloader = torch.utils.data.DataLoader(alex_trainset, batch_size=batchsize, pin_memory=True,
                                          shuffle=True)

alex_testset = torchvision.datasets.CIFAR10(root='/tmp/data', train=False,
                                       download=True, transform=transformAlexTest)
alex_testloader = torch.utils.data.DataLoader(alex_testset, batch_size=batchsize, pin_memory=True,
                                         shuffle=True)


Commençons par collecter le réseau entraîné et étudions sa structure.

Que faut-il modifier pour l'adapter à notre cas ? En outre on aimerait que lors de l'apprentissage seuls les poids des modules modifiés soient ajustés. Penser à fixer les autres.

In [ ]:
from torchvision import models
alexnet = models.alexnet(weights=True)
alexnet.name = "alexnet"+time.asctime()
print(alexnet)


set_parameter_requires_grad(alexnet,False)
# Pour avoir 10 classes et non 1000 en sortie :
num_ftrs = alexnet.classifier[6].in_features
alexnet.classifier[6] = nn.Linear(num_ftrs,len(cifar_trainset.classes)) # modification de la dernière couche


## Fine-Tuning d'AlexNet

Faites le Fine-tuning de alexnet sur les données CIFAR. Regardez les cartes de saillances obtenues.

In [ ]:
train(alexnet,10,alex_trainloader,alex_testloader)


In [ ]:
# sauvegarde / chargement

path = "./model/"

# fichier = path+f"myAlexNet"
# save_model(alexnet,fichier)


# vous pouvez utiliser les formules symmétriques pour le chargement
# seule maniere de recreer le réseau :
alexnet = models.alexnet(weights=True)
alexnet.name = "alexnet"+time.asctime()
set_parameter_requires_grad(alexnet,False)
# Pour avoir 10 classes et non 1000 en sortie :
num_ftrs = alexnet.classifier[6].in_features
alexnet.classifier[6] = nn.Linear(num_ftrs,len(cifar_trainset.classes)) # modification de la dernière couche
alexnet = alexnet.to(device)

load_model( path+f"myAlexNet", alexnet)


In [ ]:
# verification performances

# x,y = next(iter(alex_testloader))
# x,y = x.to(device), y.to(device)
# print(x.size())
# yhat = alexnet(x)
# print(accuracy(yhat, y))

# pour toutes les données
print("alexNet : ", compute_accuracy(alexnet, alex_testloader))

# check
# alexNet :  tensor(0.8289, device='mps:0')

In [ ]:
## Carte de saillance du réseau

inputs,labels=next(iter(alex_testloader))
for i in range(len(cifar_trainset.classes)):
  print("Pour ",cifar_trainset.classes[i])
  getSaliency(alexnet,inputs[0],i)


In [ ]:
## Remplacement de la couche classifier par un module d'average pooling et un linéaire.
class View(nn.Module):
    def __init__(self,shape):
        super(View, self).__init__()
        self.shape=shape

    def forward(self, x):
        return x.view(self.shape)
     
    def extra_repr(self):
        return str(self.shape)      


# Fixons d'abord les poids du réseau :
set_parameter_requires_grad(alexnet,False)
      
alexnet.classifier = torch.nn.Sequential(torch.nn.Unflatten(1,(256,6,6)),
                    torch.nn.AvgPool2d(6,1),
                    torch.nn.Flatten(1),torch.nn.Linear(256, 10))
# Car AlexNet fait un flatten entre la partie features et la partie classifieur:
#alexnet.classifier.add_module("view", View((#ANSWER[[ \ 
#                                          -1,256, 6 , 6\
#                                        #]]ANSWER \
#                                      ))) 

#alexnet.classifier.add_module("avgPool", torch.nn.AvgPool2d(6, 1))
#alexnet.classifier.add_module("view2", View((-1,256)))
#alexnet.classifier.add_module("Linear", torch.nn.Linear(256, 10))

print(alexnet)


In [ ]:
## Entrainement du réseau
train(alexnet,5,alex_trainloader,alex_testloader)


In [ ]:
inputs,labels=iter(alex_trainloader).next()
generate_cam(alexnet,inputs[0].unsqueeze(0))

On peut aussi charger des images du web et voir ce que notre classifieur donne. Par exemple:

In [ ]:
# !wget "https://www.fidanimo.com/sites/default/files/2020-10/dog-sitter.jpg"
# !wget "https://upload.wikimedia.org/wikipedia/commons/a/aa/L%27_arrivée_d%27un_navire_cargo_%282%29.jpg"
from PIL import Image
path = "./data/"

size = 32, 32
im = Image.open(path+"dog-sitter.jpg")
im = im.resize(size, Image.Resampling.LANCZOS) # pour rendre les images compatibles
im.save(path+"dog-sitter-r.jpg", "JPEG")
im = Image.open(path+"cargo.jpg")
im = im.resize(size, Image.Resampling.LANCZOS)
im.save(path+"cargo-r.jpg", "JPEG")


In [ ]:

imageDog = transformAlexTest(Image.open(path+"dog-sitter-r.jpg")).unsqueeze(0).to(device, torch.float)
imageShip = transformAlexTest(Image.open(path+"cargo-r.jpg")).unsqueeze(0).to(device,torch.float)


print(imageDog.size())


In [ ]:
print(alexnet(imageDog))
print(alexnet(imageShip))
print(cifar_trainset.classes)

# Construction du sujet à partir de la correction

In [1]:
### <CORRECTION> ###
import re
# transformation de cet énoncé en version étudiante

fname = "2_3-CNN-corr.ipynb" # ce fichier
fout  = fname.replace("-corr","")

# print("Fichier de sortie: ", fout )

f = open(fname, "r")
txt = f.read()
 
f.close()


f2 = open(fout, "w")
f2.write(re.sub("<CORRECTION>.*?(</CORRECTION>)"," TODO ",\
    txt, flags=re.DOTALL))
f2.close()

### </CORRECTION> ###